# Embedded Bokeh Server

* Docs: https://docs.bokeh.org/en/latest/docs/user_guide/server.html#embedding-bokeh-server-as-a-library

* Stackoverflow: https://stackoverflow.com/questions/54069272/how-to-run-a-bokeh-server-without-bokeh-server-show-command
* args:
https://github.com/bokeh/bokeh/blob/branch-3.0/bokeh/command/subcommands/serve.py


In [ ]:
# bokeh serve --show app --allow-websocket-origin server.local:5006


In [8]:
%%writefile bkapp.py

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature
from bokeh.server.server import Server
from bokeh.themes import Theme


def bkapp(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25), y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    # doc.theme = Theme(filename="theme.yaml")

# Setting num_procs here means we can't touch the IOLoop before now, we must
# let Server handle that. If you need to explicitly handle IOLoops then you
# will need to use the lower level BaseServer class.
import os
print(f'start DNS_NAME = {os.getenv("DNS_NAME")}')
server = Server({'/': bkapp}, num_procs=1, port=5007, allow_websocket_origin= [ "server.local:5007" ])
server.start()

if __name__ == '__main__':
    print('Opening Bokeh application on http://server.local:5007/')

    server.io_loop.add_callback(server.show, "/")
    server.io_loop.start()


Overwriting bkapp.py
